In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

  Obtaining dependency information for google-cloud-pipeline-components>2 from https://files.pythonhosted.org/packages/29/05/e2b13e7982506e6f63c2973c7a139fd8905dfd25ba5a7e03a0cb4541a76d/google_cloud_pipeline_components-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/f6/67/734b8c73b8e708a24301b8a0a072ddfe936816896d12af4884e4f7bbd3b0/google_cloud_aiplatform-1.35.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 175.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [25]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,ModelDeployOp)
from google_cloud_pipeline_components.types import artifact_types

In [2]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "data-engineering-lab-1"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_group11"

Component: Data Ingestion

In [3]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

Component: Feature selection

In [4]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def feature_selection(dataset: Input[Dataset], dataset_train_x: Output[Dataset], dataset_train_y: Output[Dataset], dataset_test_y: Output[Dataset], dataset_test_x: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    complete_data = pd.read_csv(dataset.path, index_col=None)
    
    label_encoder = LabelEncoder()
    for i in ['Geography', 'Gender']:
        complete_data[i]= label_encoder.fit_transform(complete_data[i])
    
    X_Columns = ['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']
    Y_columns= ['Exited']
    
    X = complete_data.copy()[X_Columns]
    y = complete_data.copy()[Y_columns]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    stratify=y)
    
    X_train.to_csv(dataset_train_x.path + ".csv" , index=False, encoding='utf-8-sig')
    y_train.to_csv(dataset_train_y.path + ".csv" , index=False, encoding='utf-8-sig')
    X_test.to_csv(dataset_test_x.path + ".csv" , index=False, encoding='utf-8-sig')
    y_test.to_csv(dataset_test_y.path + ".csv" , index=False, encoding='utf-8-sig')

Component: Training RF model

In [5]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn", "pickle"],
    base_image="python:3.10.7-slim"
)
def train_rfc(features_x: Input[Dataset], features_y: Input[Dataset], out_model: Output[Model]):
    '''train a MLP with default parameters'''
    import pandas as pd
    import json
    import logging 
    import sys
    import os
    import pickle
    from sklearn.ensemble import RandomForestClassifier
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df_x = pd.read_csv(features_x.path+".csv")
    df_y = pd.read_csv(features_y.path+".csv")
    
    logging.info(df_x.columns)
        
    # define model
    model_rfc = RandomForestClassifier()
    # fit model
    model_rfc.fit(df_x, df_y)

    #save the model
    model_rfc.metadata["framework"] = "RFC"
    file_name = model_rfc.path + ".sav"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_rfc, file)   

Component: Evaluate model performance

In [6]:
@dsl.component(
    packages_to_install = [
       "pandas", "scikit-learn", "numpy", "pickle"], 
    base_image="python:3.10.7-slim"
)

def rfc_model_evaluation(test_set_x:  Input[Dataset], test_set_y:  Input[Dataset], model_rfc: Input[Model], kpi: Output[Metrics]):
    import pandas as pd
    import logging     
    from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score
    import json
    import pickle
    from numpy import nan_to_num
    from sklearn.metrics import accuracy_score
    
    def threshold_check(val1, val2):
        cond = False
        if val1 >= val2 :
            cond = True
        return cond

    data_x = pd.read_csv(test_set_x.path+".csv")
    data_y = pd.read_csv(test_set_y.path+".csv")
    
    #Loading the saved model with joblib
    m_filename = model_rfc.path + ".sav"
    model = pickle.load(open(m_filename, 'rb'))
    
    predictions_rfc = model_rfc.predict(data_x)
    accuracy = accuracy_score(data_y, predictions_rfc)
    
    model_rfc.metadata["accuracy"] = float(accuracy)
    kpi.log_metric("accuracy", float(accuracy))

Upload model, metrics to google buckets

In [7]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob('model.sav')
    source_file_name= model.path + '.sav'
   
    blob.upload_from_filename(source_file_name)    
    
    print(f"File {source_file_name} uploaded to {model_repo}.")

Defining the pipeline

In [8]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="churn-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, file_name: str, dataset_uri: str, model_repo: str, model_repo_uri:str):    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=file_name
    )
     
    model_feature_selection = feature_selection(dataset=di_op.output)
        
    model_train = train_rfc(features_x=model_feature_selection.outputs["dataset_train_x"], features_y=model_feature_selection.outputs["dataset_train_y"])
    
    model_evaluation = rfc_model_evaluation(
        test_set_x=model_feature_selection.outputs["dataset_test_x"],
        test_set_y=model_feature_selection.outputs["dataset_test_y"],
        model_rfc=model_train.output,
    )
    

    upload_model_to_gc_op = upload_model_to_gcs(
        project_id=project_id,
        model_repo=model_repo,
        model=model_train.output
    )    
        
    import_unmanaged_model_task = dsl.importer(
        artifact_uri=model_repo_uri,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",  # see https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers  
            },
        },
    ).after(upload_model_to_gc_op)      
       
    # using Google's custom components for for uloading and deploying the model.
       
    model_upload_op = ModelUploadOp(
        project=project_id,
        display_name="churn-prediction-model",
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    ).after(import_unmanaged_model_task)       
               
    create_endpoint_op = EndpointCreateOp(
        project=project_id,
        display_name="churn-prediction-service",
    ).after(model_upload_op)      
        
    model_deploy_op = ModelDeployOp(
        model=model_upload_op.outputs["model"],
        endpoint=create_endpoint_op.outputs['endpoint'],
        deployed_model_display_name="churn-prediction-model",
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        traffic_split={"0": 100},
    ).after(create_endpoint_op) 

In [9]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='churn_predictor_training_pipeline.yaml')

In [10]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="churn-predictor",
    enable_caching=False,
    template_path="churn_predictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': 'data-engineering-lab-1', # makesure to use your project id 
        'data_bucket': 'data_group11',  # makesure to use your data bucket name 
        'dataset_uri':'gs://data_group11/training_set.csv',
        'model_repo':'models_group11', # makesure to use your model bucket name 
        'model_repo_uri':'gs://models_group11' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob


InvalidArgument: 400 Some input parameters of the PipelineSpec.root are missing both defaultValue and default value from PipelineJob.runtimeConfig.parameter_values: ([file_name])